In [20]:
(defn gte-in-words [exp1 exp2]
    (str "is every element in " (exp1 :right) " greater than or equal to " exp2
         " and is " exp1 " greater than or equal to every element in " (exp2 :left)))

#'user/gte-in-words

In [21]:
(def zero {:left :nullset :right :nullset})

#'user/zero

In [22]:
(gte-in-words zero zero)

"is every element in :nullset greater than or equal to {:left :nullset, :right :nullset} and is {:left :nullset, :right :nullset} greater than or equal to every element in :nullset"

In [30]:
(defn gte-vacuously-true [exp1 exp2]
    (let [exp1-is-empty (= :nullset (exp1 :left))
          exp2-is-empty (= :nullset (exp2 :right))]
        (and exp1-is-empty exp2-is-empty)))

#'user/gte-vacuously-true

test case 1

In [31]:
(gte-vacuously-true zero zero)

true

In [27]:
(def one {:left zero :right :nullset})

#'user/one

next test case

In [32]:
(gte-vacuously-true one zero)

false

Let's make a shorthand that can easily expand left and right when using the empty set:

In [45]:
(defn expand-simple-empty-set [thing]
    (cond 
        (and (not (thing :left)) (not (thing :right)))
        zero
        (not (thing :right)) (assoc thing :right :nullset)
        (not (thing :left)) (assoc thing :left :nullset)
        :else thing))

#'user/expand-simple-empty-set

In [46]:
(expand-simple-empty-set {})

{:left :nullset, :right :nullset}

In [47]:
(expand-simple-empty-set {:left 8})

{:left 8, :right :nullset}

In [48]:
(expand-simple-empty-set {:right "asdf"})

{:right "asdf", :left :nullset}

In [49]:
(expand-simple-empty-set {:left 8 :right 19})

{:left 8, :right 19}

Let's try answering the question for just numbers of the form $\{n|\}$ or $\{|n\}$ where $n$ is an integer or when the expression is $\{|\}$.  First, let's detect whether it is the case:

In [50]:
(defn is-simple-integer-form [exp]
    (let [expanded (expand-simple-empty-set exp)
          e-left (expanded :left)
          e-right (expanded :right)
          l-is-empty (= :nullset e-left)
          r-is-empty (= :nullset e-right)
          l-is-int (integer? e-left)
          r-is-int (integer? e-right)]
        (or (and l-is-empty r-is-int)
            (and r-is-empty l-is-int)
            (and l-is-empty r-is-empty))))

#'user/is-simple-integer-form

In [51]:
(is-simple-integer-form zero)

true

In [53]:
(is-simple-integer-form {:left 1 :right "potato"})

false

In [54]:
(is-simple-integer-form {:left "asdf" :right 3})

false

In [55]:
(is-simple-integer-form {:left 1})

true

In [57]:
(is-simple-integer-form {:right -100})

true

In [58]:
(is-simple-integer-form {:left 0 :right 1})

false

That last one is important -- we don't want to handle fractional cases yet, or anything other than the expected form.  Now let's resolve simple integer forms to straight integers:

In [89]:
(defn basic-integer-form-to-int [exp]
    (if (is-simple-integer-form exp)
        (let [expanded (expand-simple-empty-set exp)]
            (cond (= zero expanded) 0
                  (= :nullset (expanded :left)) (- (expanded :right) 1)
                  :else (+ (expanded :left) 1)))))

#'user/basic-integer-form-to-int

In [91]:
(basic-integer-form-to-int {:left 1})

2

In [90]:
(basic-integer-form-to-int {:right -100})

-101

Let's try gte for simple integer forms:

In [ ]:
(defn gte-simple-integer-form [exp1 exp2]
    (if (and (is-simple-integer-form exp1) (is-simple-integer-form exp2))
        (let [e1-right (exp1 :right)
              e2-left (exp2 :left)]
            ;; to be continued)
        (str "hey what is this " exp1 " &&& " exp2 " you gave me? At least one of them is not a simple integer form")))